In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Concatenate
from keras.layers.core import Dense, Reshape, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization

In [8]:
def create_model(embed_size=128, max_length=300, filter_sizes=(2,3,4,5), filter_num=64, dropout=0.5):
  inp = Input(shape=(max_length,))
  emb = Embedding(0xffff, embed_size)(inp)
  emb_ex = Reshape((max_length, embed_size, 1))(emb)
  convs = []
  
  for filter_size in filter_sizes:
    conv = Conv2D(filter_num, (filter_size, embed_size), activation="relu")(emb_ex)
    pool = MaxPooling2D(pool_size=(max_length - filter_size + 1, 1))(conv)
    convs.append(pool)
    
  convs_merged = Concatenate()(convs)
  reshape = Reshape((filter_num * len(filter_sizes), ))(convs_merged)
  fc1 = Dense(64, activation="relu")(reshape)
  bn1 = BatchNormalization()(fc1)
  do1 = Dropout(dropout)(bn1)
  fc2 = Dense(1, activation='sigmoid')(do1)
  model = Model(inputs=inp, outputs=fc2)
  return model

In [9]:
create_model()